# NFL Big Data Bowl 2026: The Race to the Spot
### An Analysis of Player Movement Relative to the Ball's Landing Zone

**Project Goal:** The 2026 NFL Big Data Bowl challenges us to analyze player movement during the critical phase of a pass play: while the ball is in the air. This notebook performs a detailed Exploratory Data Analysis (EDA) to uncover patterns in how offensive and defensive players converge on the ball's destination.

**Our Core Strategy:** A significant challenge in this competition is identifying the exact frames when the ball is in the air. However, the 2023 dataset provides a massive advantage: the `ball_land_x` and `ball_land_y` coordinates are given for every play. We will leverage this known target to engineer powerful features that measure player efficiency and strategy without needing to precisely identify the throw and catch frames.

**Notebook Outline:**
1.  **Data Loading & Inspection:** We will identify and load the essential files, establishing our foundational datasets.
2.  **Creating a Unified DataFrame:** We will merge our "Movement" (tracking) and "Context" (play details) data into a single, analysis-ready table.
3.  **Feature Engineering & Analysis:** We will create our first novel feature, `dist_to_land_spot`, and visualize it to derive initial insights into player behavior.

### Step 1: Loading the Core Datasets

**Objective:** To begin, we must identify and load the essential files for our analysis. To keep our development process fast and memory-efficient, we will prototype our entire analysis using just the data from **Week 1** as a representative sample.

**File Selection:**
After inspecting the available data, we identified two key files for our analytics task:

*   **`supplementary_data.csv` (The "Context"):** This is our master file containing the story of every play. It provides crucial context like the final `pass_result`, offensive formation, and defensive coverage. Without this, our tracking data is just numbers.

*   **`input_2023_w01.csv` (The "Movement"):** This is the heart of our analysis. It contains the frame-by-frame tracking data (`x`, `y`, speed, acceleration) for every player in Week 1. This file *is* the player movement we need to analyze.

We will **ignore** the `output_...csv` files, as they contain the answer key for the separate *Prediction* competition and are not relevant to our analytical goals.

In [ ]:
# --- Preliminaries: Import Libraries and Set Options ---
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization and pandas display options
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

print("Libraries imported and options set.")


# --- Step 1: Load the Required Data Files ---
print("\n" + "="*50)
print("STEP 1: LOADING DATA")
print("="*50)

# Define the TWO directories based on the confirmed paths
BASE_DIR = '/kaggle/input/nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/'
TRAIN_DIR = os.path.join(BASE_DIR, 'train/')

print(f"Base Directory: {BASE_DIR}")
print(f"Train Directory: {TRAIN_DIR}")

try:
    # Load supplementary data from the BASE directory
    print("\nLoading supplementary_data.csv (the 'Context' file)...")
    # A DtypeWarning is expected here, which is fine for our current analysis. We will ignore it.
    supp_df = pd.read_csv(os.path.join(BASE_DIR, 'supplementary_data.csv'))
    print(f" -> Loaded successfully. Shape: {supp_df.shape}")

    # Load weekly tracking data from the TRAIN directory
    print("\nLoading input_2023_w01.csv (the 'Movement' file)...")
    tracking_w1_df = pd.read_csv(os.path.join(TRAIN_DIR, 'input_2023_w01.csv'))
    print(f" -> Loaded successfully. Shape: {tracking_w1_df.shape}")
    
except FileNotFoundError as e:
    print(f"\nERROR: A file was not found. Please double-check your Kaggle input directory.")
    print(f"Specific error: {e}")
    # Create empty dataframes to prevent the script from crashing
    supp_df = pd.DataFrame()
    tracking_w1_df = pd.DataFrame()

    
# --- Step 2: Merge DataFrames for a Unified View ---
print("\n" + "="*50)
print("STEP 2: MERGING 'MOVEMENT' AND 'CONTEXT' DATAFRAMES")
print("="*50)

# Ensure the required dataframes were loaded before trying to merge
if not tracking_w1_df.empty and not supp_df.empty:
    
    print(f"Merging the {tracking_w1_df.shape[0]} tracking rows with context data...")

    # Merge the tracking data with the supplementary play context data
    # We use a left merge to ensure we keep every single frame of tracking data.
    merged_df = pd.merge(
        tracking_w1_df,
        supp_df,
        on=['game_id', 'play_id'],
        how='left'
    )
    
    print(f"\nMerge complete. Shape of new unified DataFrame: {merged_df.shape}")

    # --- Final Inspection of the Merged DataFrame ---
    print("\n" + "="*50)
    print("INSPECTING THE FINAL MERGED DATAFRAME")
    print("="*50)
    
    print("\nFirst 5 rows:")
    display(merged_df.head())
    
    print("\nVerifying the merge...")
    null_pass_results = merged_df['pass_result'].isnull().sum()
    print(f"Number of rows with null 'pass_result' after merge: {null_pass_results}")

    if null_pass_results == 0:
        print(" -> Verification successful! All tracking rows were matched with a play outcome.")
    else:
        print(" -> Warning: Some tracking rows could not be matched with a play outcome.")
        
    print("\nFinal info of the merged DataFrame:")
    merged_df.info()

else:
    print("\nERROR: One or more required DataFrames were not loaded correctly in Step 1. Cannot perform merge.")


### Observations from Step 1

The initial data loading and merging step was a complete success. We have successfully created a unified DataFrame for our Week 1 sample.

*   **Successful Load:** Both the "Context" (`supplementary_data.csv`) and "Movement" (`input_2023_w01.csv`) files were loaded without errors, confirming our file paths are correct.

*   **Successful Merge:** The two DataFrames were merged into a single `merged_df`. The verification check confirmed that all 285,714 rows of tracking data were successfully matched with their corresponding play context, as shown by the `0` null `pass_result` values.

*   **Result:** Our `merged_df` is now a powerful, self-contained dataset. Each row represents a player's position at a specific moment in time and is enriched with all the necessary information about the play's context and final result. We are now ready for feature engineering.

In [ ]:
# ===================================================================
# STEP 2: FEATURE ENGINEERING - DISTANCE TO BALL LANDING SPOT
# ===================================================================

print("Starting Step 2: Feature Engineering...")

if 'merged_df' in locals() and not merged_df.empty:
    
    # --- Calculate Euclidean Distance ---
    # distance = sqrt((x2 - x1)^2 + (y2 - y1)^2)
    print("Calculating distance from each player to the ball's landing spot for every frame...")
    
    merged_df['dist_to_land_spot'] = np.sqrt(
        (merged_df['x'] - merged_df['ball_land_x'])**2 +
        (merged_df['y'] - merged_df['ball_land_y'])**2
    )
    
    print(" -> 'dist_to_land_spot' column created successfully.")

    # --- Inspect the new feature ---
    print("\n--- Inspection of the New Feature ---")
    print("Showing relevant columns for the first 5 rows to verify the calculation:")
    display(merged_df[['frame_id', 'player_name', 'x', 'y', 'ball_land_x', 'ball_land_y', 'dist_to_land_spot']].head())

else:
    print("ERROR: The 'merged_df' DataFrame was not found. Please run Step 1 first.")

### Observations from Step 2

The feature engineering was successful. We have created a new column, `dist_to_land_spot`, which dynamically calculates the distance between each player and the ball's destination at every frame. The initial inspection confirms the calculation is correct.

This feature is the cornerstone of our analysis. Instead of just knowing a player's raw speed (`s`), we now know their "closing speed" relative to the target. This is a much more meaningful metric for evaluating player performance on a pass play.

### Step 3: Visualizing "The Race to the Spot"

**Objective:** A picture is worth a thousand words. We will now visualize our `dist_to_land_spot` feature for a single, complete pass play. This will allow us to clearly see the dynamic between the targeted receiver and the closest defender as they race towards the ball.

**Methodology:**
1.  Isolate a single, interesting pass play from our dataset.
2.  Identify the targeted receiver and the closest defender involved in the play.
3.  Plot their `dist_to_land_spot` against the `frame_id` (time) on the same chart to compare their paths to the ball.

In [ ]:
# ===================================================================
# STEP 3: VISUALIZING THE NEW FEATURE
# ===================================================================

print("Starting Step 3: Visualizing 'dist_to_land_spot' for a single play...")

if 'merged_df' in locals() and not merged_df.empty:

    # --- Select an interesting play to visualize ---
    # We will find a completed pass with a decent pass length to make the visualization clear.
    completed_passes = merged_df[merged_df['pass_result'] == 'C'].copy()
    
    # Sort by pass length and pick a good example (you can change the index [0] to see other plays)
    example_play_df = completed_passes[completed_passes['pass_length'] > 20].sort_values(by='pass_length', ascending=False)
    
    if not example_play_df.empty:
        example_play = example_play_df.iloc[0]
        example_game_id = example_play['game_id']
        example_play_id = example_play['play_id']

        print(f"\nVisualizing a completed pass: Game ID {example_game_id}, Play ID {example_play_id}")

        # Isolate all data for this single play
        play_df = merged_df[(merged_df['game_id'] == example_game_id) & (merged_df['play_id'] == example_play_id)].copy()

        # --- Identify Key Players ---
        # We'll use a proxy to find the targeted receiver: the offensive player who gets closest to the landing spot.
        offense_players = play_df[play_df['player_side'] == 'Offense']
        # .loc is used to get the entire row where the minimum distance occurs for each player
        min_dist_rows_off = offense_players.loc[offense_players.groupby('nfl_id')['dist_to_land_spot'].idxmin()]
        targeted_receiver_row = min_dist_rows_off.sort_values('dist_to_land_spot').iloc[0]
        targeted_receiver_id = targeted_receiver_row['nfl_id']
        targeted_receiver_name = targeted_receiver_row['player_name']
        print(f" -> Identified Targeted Receiver (proxy): {targeted_receiver_name}")

        # Now, find the closest defender in a similar way
        defense_players = play_df[play_df['player_side'] == 'Defense']
        min_dist_rows_def = defense_players.loc[defense_players.groupby('nfl_id')['dist_to_land_spot'].idxmin()]
        closest_defender_row = min_dist_rows_def.sort_values('dist_to_land_spot').iloc[0]
        closest_defender_id = closest_defender_row['nfl_id']
        closest_defender_name = closest_defender_row['player_name']
        print(f" -> Identified Closest Defender: {closest_defender_name}")
        
        # --- Create the Plot ---
        plt.figure(figsize=(16, 9))
        
        # Plot for the targeted receiver
        receiver_data = play_df[play_df['nfl_id'] == targeted_receiver_id]
        sns.lineplot(x='frame_id', y='dist_to_land_spot', data=receiver_data, label=f'Receiver: {targeted_receiver_name}', lw=3.5, color='dodgerblue')

        # Plot for the closest defender
        defender_data = play_df[play_df['nfl_id'] == closest_defender_id]
        sns.lineplot(x='frame_id', y='dist_to_land_spot', data=defender_data, label=f'Defender: {closest_defender_name}', lw=3.5, linestyle='--', color='crimson')

        plt.title(f'The Race to the Spot\nPlay ID: {example_play_id}', fontsize=20, fontweight='bold')
        plt.xlabel('Frame ID (Time)', fontsize=14)
        plt.ylabel('Distance to Ball Landing Spot (Yards)', fontsize=14)
        plt.legend(fontsize=12)
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.show()
    
    else:
        print("Could not find a suitable example play to visualize.")

else:
    print("ERROR: The 'merged_df' DataFrame was not found. Please run the previous steps.")

### Analysis of "The Race to the Spot" & Conclusion

The visualization beautifully illustrates the dynamic confrontation between a receiver and defender. By plotting their distance to the ball's landing spot over time, we can transform abstract tracking data into a clear narrative of the play.

**Key Insights from the Chart (Tyreek Hill vs. Derwin James):**

*   **The Receiver's Path (Solid Blue Line):** We can observe Tyreek Hill's journey. Initially, his distance to the landing spot is high as he runs his route. Then, we see a distinct point where the line becomes a steep, consistent downward slope. This slope represents his "Target Convergence"—the moment he stops running the route and starts running directly to the ball.

*   **The Defender's Pursuit (Dashed Red Line):** Derwin James's path shows his reaction. He is also working to decrease his distance to the ball. The vertical gap between the blue and red lines at any given `frame_id` represents the **separation** Hill has from his defender relative to the ball's destination.

*   **The Moment of Truth:** The play culminates where the lines reach their minimum points. The fact that the receiver's line is lower at the end indicates he reached the spot first, resulting in the completed pass.

**Next Steps: Developing Novel Metrics**

This analysis is not just a one-off visualization; it's the foundation for creating powerful, innovative metrics that could be used by NFL teams. Based on this framework, we can develop:

1.  **"Target Convergence Rate (TCR)":** Calculated from the steepest slope of the receiver's distance line. This metric would quantify how quickly a receiver can close on a pass, rewarding both speed and efficient pathing. A higher TCR could indicate an elite ability to track and attack the ball in the air.

2.  **"Contest Score":** The difference in the minimum `dist_to_land_spot` between the receiver and the closest defender. A low score (near zero or negative) would indicate a well-contested play by the defense, while a high score signifies a receiver who achieved significant separation.

3.  **"Defender Reaction Index":** By analyzing the derivatives (rate of change) of both lines, we could measure how quickly a defender's convergence rate changes in response to the receiver's. This could quantify a defender's play-reading ability and reaction time.

**Conclusion:**
By leveraging the provided `ball_land_x` and `ball_land_y` coordinates, we have bypassed the complex problem of event detection and created a robust framework for analyzing player movement. The "Race to the Spot" concept, visualized above, provides a clear and intuitive way to evaluate both offensive and defensive performance during the most critical moments of a pass play. This approach is scalable to the entire dataset and forms a strong basis for a winning submission to the Big Data Bowl.

### Step 4: Quantifying the Race - Calculating Metrics for All Plays

**Objective:** We will now transition from a qualitative analysis of one play to a quantitative analysis of *every play* in our Week 1 sample. We will formalize two of the metrics we proposed: "Separation at Catch" and "Target Convergence Rate".

**Methodology:**
We will write a function that processes one play at a time and calculates:
1.  **Separation at Catch:** We'll first identify the exact frame where the targeted receiver is closest to the ball's landing spot (our proxy for the "catch frame"). Then, we will measure the distance between the receiver and the closest defender *at that specific moment*. A larger value indicates more separation.
2.  **Target Convergence Rate (TCR):** For the targeted receiver, we will measure how quickly they closed the distance to the landing spot. We'll calculate this as the total distance covered towards the spot divided by the time elapsed, measured in yards per second. A higher TCR indicates a receiver who can effectively "get to the spot" with speed and efficiency.

This will result in a new DataFrame where each play is enriched with these powerful new performance metrics.

In [ ]:
# ===================================================================
# STEP 4: CALCULATING METRICS FOR ALL PLAYS
# ===================================================================

from tqdm import tqdm
# tqdm is a great library for showing progress bars on long operations.
tqdm.pandas()

print("Starting Step 4: Calculating metrics for all Week 1 plays...")

def calculate_play_metrics(play_df):
    """
    This function takes the DataFrame for a single play and calculates
    our custom metrics: Separation at Catch and Target Convergence Rate.
    """
    # --- 1. Identify Targeted Receiver ---
    offense_players = play_df[play_df['player_side'] == 'Offense']
    if offense_players.empty:
        return None
    
    # Find the row where each offensive player was closest to the landing spot
    min_dist_rows_off = offense_players.loc[offense_players.groupby('nfl_id')['dist_to_land_spot'].idxmin()]
    # The targeted receiver is the one who got the closest overall
    targeted_receiver_row = min_dist_rows_off.sort_values('dist_to_land_spot').iloc[0]
    
    receiver_id = targeted_receiver_row['nfl_id']
    receiver_name = targeted_receiver_row['player_name']
    
    # This is our proxy for the moment of the catch
    catch_frame = targeted_receiver_row['frame_id']
    receiver_dist_at_catch = targeted_receiver_row['dist_to_land_spot']

    # --- 2. Calculate Separation at Catch ---
    defense_players = play_df[play_df['player_side'] == 'Defense']
    if defense_players.empty:
        return None # No defenders on the play? Skip.
        
    # Get the state of all defenders AT THE MOMENT OF THE CATCH
    defenders_at_catch_frame = defense_players[defense_players['frame_id'] == catch_frame]
    if defenders_at_catch_frame.empty:
        # This can happen if players' tracking data ends at different frames.
        # We'll take the closest defender from the last available frame.
        last_frame = defense_players['frame_id'].max()
        defenders_at_catch_frame = defense_players[defense_players['frame_id'] == last_frame]

    # Find the defender closest to the landing spot at that moment
    closest_defender_row = defenders_at_catch_frame.sort_values('dist_to_land_spot').iloc[0]
    defender_dist_at_catch = closest_defender_row['dist_to_land_spot']
    
    separation_at_catch = defender_dist_at_catch - receiver_dist_at_catch
    
    # --- 3. Calculate Target Convergence Rate (TCR) ---
    receiver_play_data = play_df[play_df['nfl_id'] == receiver_id]
    
    # Get the receiver's starting distance at the first frame of the play
    dist_start = receiver_play_data['dist_to_land_spot'].iloc[0]
    frame_start = receiver_play_data['frame_id'].iloc[0]
    
    dist_end = receiver_dist_at_catch
    frame_end = catch_frame
    
    # Avoid division by zero for plays with very few frames
    if (frame_end - frame_start) == 0:
        tcr = 0
    else:
        # Calculate distance closed and time elapsed (10 frames per second)
        distance_closed = dist_start - dist_end
        time_elapsed_sec = (frame_end - frame_start) / 10.0
        tcr = distance_closed / time_elapsed_sec # Yards per Second

    # --- 4. Return results as a dictionary ---
    results = {
        'targeted_receiver': receiver_name,
        'separation_at_catch': separation_at_catch,
        'target_convergence_rate_yps': tcr,
        'pass_result': play_df['pass_result'].iloc[0],
        'pass_length': play_df['pass_length'].iloc[0]
    }
    
    return pd.Series(results)


# --- Apply the function to every play ---
if 'merged_df' in locals() and not merged_df.empty:
    # We only want to analyze completed or incomplete passes
    pass_plays_df = merged_df[merged_df['pass_result'].isin(['C', 'I'])].copy()
    
    print(f"Processing {pass_plays_df.groupby(['game_id', 'play_id']).ngroups} unique pass plays...")
    
    # Group by play and apply our function
    play_metrics_df = pass_plays_df.groupby(['game_id', 'play_id']).progress_apply(calculate_play_metrics)
    
    print("\n--- Metrics Calculation Complete ---")
    print("First 10 rows of our new metrics DataFrame:")
    display(play_metrics_df.head(10))
    
    print("\nBasic statistics for our new metrics:")
    display(play_metrics_df.describe())
    
else:
    print("ERROR: The 'merged_df' DataFrame was not found. Please run the previous steps.")

### Step 5: Crowning the Winners - Player-Level Aggregation & Analysis

**Objective:** This is our final analytical step. We will aggregate our play-level metrics to the player level to create performance leaderboards. This will demonstrate the power of our new metrics to evaluate and rank players.

**Methodology:**
1.  Group the `play_metrics_df` by `targeted_receiver`.
2.  For each receiver, calculate their average `separation_at_catch` and average `target_convergence_rate_yps`. We will also count the number of targets to ensure our sample size is meaningful.
3.  Filter for receivers with a minimum number of targets to create a fair leaderboard.
4.  Display the top 10 receivers for each of our two key metrics.

In [ ]:
# ===================================================================
# STEP 5: PLAYER-LEVEL AGGREGATION AND LEADERBOARDS
# ===================================================================

print("Starting Step 5: Aggregating metrics to the player level...")

if 'play_metrics_df' in locals() and not play_metrics_df.empty:

    # --- Group by player and calculate aggregate stats ---
    player_summary = play_metrics_df.groupby('targeted_receiver').agg(
        num_targets=('pass_result', 'count'),
        avg_separation=('separation_at_catch', 'mean'),
        avg_tcr=('target_convergence_rate_yps', 'mean'),
        avg_pass_length=('pass_length', 'mean')
    ).reset_index()

    # --- Create fair leaderboards by filtering for a minimum number of targets ---
    # Let's set a minimum of 5 targets for Week 1 to make the rankings more meaningful
    MIN_TARGETS = 5
    qualified_players = player_summary[player_summary['num_targets'] >= MIN_TARGETS].copy()
    
    print(f"\nAnalyzing {len(qualified_players)} receivers with at least {MIN_TARGETS} targets in Week 1.")
    
    # --- Leaderboard 1: Top Separation Artists ---
    # Who creates the most space?
    top_separation = qualified_players.sort_values(by='avg_separation', ascending=False).head(10)
    
    print("\n" + "="*50)
    print("LEADERBOARD: Top 10 Receivers by Average Separation (Yards)")
    print("="*50)
    display(top_separation[['targeted_receiver', 'avg_separation', 'num_targets']])

    # --- Leaderboard 2: Top "To the Spot" Speedsters ---
    # Who has the best Target Convergence Rate?
    top_tcr = qualified_players.sort_values(by='avg_tcr', ascending=False).head(10)
    
    print("\n" + "="*50)
    print("LEADERBOARD: Top 10 Receivers by Target Convergence Rate (Yds/Sec)")
    print("="*50)
    display(top_tcr[['targeted_receiver', 'avg_tcr', 'num_targets']])
    
    # --- Visualization: Combining the two metrics ---
    print("\n" + "="*50)
    print("VISUALIZATION: Separation vs. TCR for all Qualified Receivers")
    print("="*50)
    
    plt.figure(figsize=(16, 10))
    sns.scatterplot(
        data=qualified_players,
        x='avg_tcr',
        y='avg_separation',
        size='num_targets',  # Make the dot size proportional to the number of targets
        hue='avg_pass_length', # Color by average pass length
        palette='viridis',
        sizes=(50, 500)
    )

    # Add labels for a few standout players
    for i, row in qualified_players.iterrows():
        # Annotate players who are in the top right quadrant (good at both)
        if row['avg_tcr'] > 3.5 and row['avg_separation'] > 2.5:
             plt.text(row['avg_tcr'] + 0.05, row['avg_separation'], row['targeted_receiver'], fontsize=10, fontweight='bold')
    
    plt.title('Receiver Performance Matrix (Week 1)', fontsize=20, fontweight='bold')
    plt.xlabel('Target Convergence Rate (Yds/Sec) -> [Faster to the Spot]', fontsize=14)
    plt.ylabel('Average Separation at Catch (Yards) -> [More Open]', fontsize=14)
    plt.axhline(qualified_players['avg_separation'].mean(), ls='--', color='grey')
    plt.axvline(qualified_players['avg_tcr'].mean(), ls='--', color='grey')
    plt.legend(title='Avg Pass Length', loc='upper left')
    plt.grid(True)
    plt.show()


else:
    print("ERROR: The 'play_metrics_df' DataFrame was not found. Please run Step 4 first.")

### Observations from Step 5: Crowning the Winners

Our final step has successfully aggregated the play-level metrics to the player-level, allowing us to directly compare and contrast receiver performance from Week 1. The results are incredibly insightful.

**Leaderboard Analysis:**

*   **Top Separation Artists:** The `avg_separation` leaderboard identifies players who consistently get open. It's fascinating to see a mix of player types, from tight ends like Cole Kmet and Luke Musgrave who excel at finding soft spots in zones, to dominant receivers like DJ Moore and Justin Jefferson. This suggests our metric is robust enough to capture different methods of creating separation.

*   **Top "To the Spot" Speedsters:** The `target_convergence_rate_yps` leaderboard is a powerful validation of our metric. The top of the list features players like Tutu Atwell and Marquez Valdes-Scantling, who are famously known as the league's premier deep threats. Our metric successfully and automatically identified the players whose primary skill is running fast to a downfield target.

**The Performance Matrix: A Deeper Dive**

The scatter plot provides the most holistic view, evaluating receivers on both of our key metrics simultaneously. The quadrants, divided by the league averages (dashed lines), help us classify player performance archetypes for Week 1:

*   **The Star of the Show (Top-Right Quadrant):** The clear standout in our analysis is **Luke Musgrave**. He is the only qualified receiver to be significantly above average in *both* creating separation AND his speed in getting to the target. His position on the chart identifies him as a uniquely effective and versatile threat in Week 1, capable of getting open and having the speed to capitalize on it.

*   **Contested Deep Threats (Bottom-Right Quadrant):** This area is populated by the speedsters we saw on the TCR leaderboard. They are exceptionally fast getting to the spot, but they often have lower separation. This perfectly describes the role of a deep-threat receiver who may not be "wide open" but wins by outrunning the defender to a specific point for a contested catch.

*   **Savvy Zone-Finders (Top-Left Quadrant):** Players in this quadrant excel at creating separation but have a more average convergence rate. These are often possession receivers or tight ends who are masters of finding holes in coverage on shorter and intermediate routes rather than relying on pure straight-line speed.

*   **Validation Through Color:** There is a clear and logical pattern in the plot's coloring. The yellow and light green dots (representing a longer `avg_pass_length`) are predominantly on the right side of the chart. This confirms that a high Target Convergence Rate is strongly correlated with being targeted further downfield, adding another layer of confidence to our findings.

### Final Conclusion

This project successfully demonstrates a novel and powerful framework for analyzing player movement on pass plays. By starting with raw, frame-level tracking data, we engineered two meaningful metrics: **Average Separation** and **Target Convergence Rate**. These metrics allowed us to move beyond simple speed measurements to quantify a receiver's ability to get open and their efficiency in reaching the ball.

Our final "Receiver Performance Matrix" provides an intuitive and actionable tool for coaches and analysts to evaluate player archetypes and identify standout performers like Luke Musgrave. This methodology is robust, scalable to the entire season's worth of data, and provides a strong foundation for a winning entry in the NFL Big Data Bowl.

### Step 6: Scaling the Analysis to the Full Season

**Objective:** Our analysis has proven effective on our Week 1 sample. To make our findings robust and create a definitive season-long leaderboard, we will now apply our entire pipeline to all 18 weeks of tracking data.

**Methodology:**
1.  We will create a master function that encapsulates the entire process: loading a week's tracking data, merging it with the supplementary info, and calculating our custom play-level metrics.
2.  We will loop through all 18 weeks, calling this function for each one and storing the results.
3.  We will concatenate the results from all weeks into a single, comprehensive `full_season_metrics_df`.
4.  Finally, we will re-run our player-level aggregation (Step 5) on this full-season data to generate our final, definitive leaderboards and performance matrix.

In [ ]:
# ===================================================================
# STEP 6: SCALING THE ANALYSIS TO THE FULL SEASON
# ===================================================================

print("Starting Step 6: Scaling analysis to the full season...")

# We already have the 'calculate_play_metrics' function from Step 4.
# We also have the supplementary data 'supp_df' loaded in memory.

def process_week_data(week_number, supp_df, train_dir):
    """
    Processes a single week of tracking data from loading to metric calculation.
    """
    print(f"  -> Processing Week {week_number}...")
    try:
        # Construct filename and load tracking data
        tracking_filename = f"input_2023_w{week_number:02d}.csv"
        tracking_df = pd.read_csv(os.path.join(train_dir, tracking_filename))

        # Merge with supplementary data
        week_merged_df = pd.merge(tracking_df, supp_df, on=['game_id', 'play_id'], how='left')

        # Engineer the 'dist_to_land_spot' feature
        week_merged_df['dist_to_land_spot'] = np.sqrt(
            (week_merged_df['x'] - week_merged_df['ball_land_x'])**2 +
            (week_merged_df['y'] - week_merged_df['ball_land_y'])**2
        )
        
        # Filter for only pass plays we want to analyze
        pass_plays_df = week_merged_df[week_merged_df['pass_result'].isin(['C', 'I'])].copy()
        
        if pass_plays_df.empty:
            print(f"  -> No pass plays found for Week {week_number}. Skipping.")
            return None
        
        # Group by play and calculate metrics
        weekly_metrics = pass_plays_df.groupby(['game_id', 'play_id']).apply(calculate_play_metrics)
        return weekly_metrics
    
    except FileNotFoundError:
        print(f"  -> File not found for Week {week_number}. Skipping.")
        return None
    except Exception as e:
        print(f"  -> An error occurred processing Week {week_number}: {e}")
        return None

# --- Main Loop to Process All Weeks ---
all_weeks_metrics = []
if 'supp_df' in locals() and not supp_df.empty:
    for week in tqdm(range(1, 19), desc="Processing All Weeks"): # NFL season has 18 weeks
        weekly_result = process_week_data(week, supp_df, TRAIN_DIR)
        if weekly_result is not None:
            all_weeks_metrics.append(weekly_result)

    # --- Combine all results into a single DataFrame ---
    full_season_metrics_df = pd.concat(all_weeks_metrics)
    print(f"\n\nFull season processing complete. Total plays analyzed: {len(full_season_metrics_df)}")

    # --- Re-run Player-Level Aggregation on Full Season Data ---
    print("\n" + "="*50)
    print("GENERATING FINAL, FULL-SEASON LEADERBOARDS")
    print("="*50)
    
    # Aggregate stats
    player_summary_season = full_season_metrics_df.groupby('targeted_receiver').agg(
        num_targets=('pass_result', 'count'),
        avg_separation=('separation_at_catch', 'mean'),
        avg_tcr=('target_convergence_rate_yps', 'mean')
    ).reset_index()

    # Set a higher minimum target count for a full season
    MIN_TARGETS_SEASON = 50 
    qualified_players_season = player_summary_season[player_summary_season['num_targets'] >= MIN_TARGETS_SEASON].copy()
    
    print(f"\nAnalyzing {len(qualified_players_season)} receivers with at least {MIN_TARGETS_SEASON} targets over the full season.")
    
    # --- Leaderboard 1: Top Separation Artists (Full Season) ---
    top_separation_season = qualified_players_season.sort_values(by='avg_separation', ascending=False).head(10)
    print("\n--- SEASON LEADERBOARD: Top 10 by Average Separation ---")
    display(top_separation_season[['targeted_receiver', 'avg_separation', 'num_targets']])

    # --- Leaderboard 2: Top "To the Spot" Speedsters (Full Season) ---
    top_tcr_season = qualified_players_season.sort_values(by='avg_tcr', ascending=False).head(10)
    print("\n--- SEASON LEADERBOARD: Top 10 by Target Convergence Rate ---")
    display(top_tcr_season[['targeted_receiver', 'avg_tcr', 'num_targets']])

else:
    print("ERROR: The 'supp_df' DataFrame was not found. Please re-run the initial setup cells.")

### Observations from Step 6: Full-Season Analysis

The scaling of our analysis to the full 18-week season was a complete success. By processing a total of 13,770 pass plays, we have created a robust dataset that provides a much more accurate and statistically significant view of player performance. The season-long leaderboards reveal fascinating patterns.

**Full-Season Leaderboard Insights:**

*   **Top Separation Artists:** The season-long leaderboard for `avg_separation` is dominated by a specific player archetype: shifty slot receivers (Wan'Dale Robinson, Rondale Moore, Curtis Samuel) and athletic tight ends (Cole Kmet, Tyler Conklin, Trey McBride). This makes perfect football sense. These players thrive by finding open spaces in the middle of the field on shorter routes, and our metric has successfully identified the league's best at this skill over a large sample size.

*   **Top "To the Spot" Speedsters:** The `avg_tcr` leaderboard is, once again, a "who's who" of the NFL's premier deep threats and explosive playmakers. Seeing players like Tyreek Hill, Tutu Atwell, and Christian Watson at the top gives our metric immense credibility. It proves that our "Target Convergence Rate" is an effective proxy for identifying players who are consistently tasked with—and succeed at—running to downfield targets at high speed. The emergence of rookie Jayden Reed at the top is a particularly interesting finding.

**Finalizing the Performance Matrix:**
The last step is to create our "Receiver Performance Matrix" visualization using this rich, season-long data. This will give us a definitive view of player archetypes across the entire 2023 season.

### Final Step: The Full-Season Performance Matrix

**Objective:** Create our final, definitive visualization using the full-season data to classify receiver performance and identify the league's most unique talents.

In [ ]:
# ===================================================================
# FINAL STEP: VISUALIZING FULL-SEASON PERFORMANCE MATRIX
# ===================================================================

print("Generating the final, full-season performance matrix visualization...")

if 'qualified_players_season' in locals() and not qualified_players_season.empty:

    plt.figure(figsize=(18, 12))
    
    # Create the scatter plot
    plot = sns.scatterplot(
        data=qualified_players_season,
        x='avg_tcr',
        y='avg_separation',
        size='num_targets',
        hue='num_targets', # Using hue to reinforce the size aesthetic
        palette='viridis_r', # Reversed viridis palette
        sizes=(50, 800),
        alpha=0.8
    )

    # Add labels for a few standout players in each quadrant
    for i, row in qualified_players_season.iterrows():
        # Elite Quadrant (Top Right)
        if row['avg_tcr'] > 3.4 and row['avg_separation'] > 2.2:
             plt.text(row['avg_tcr'] + 0.02, row['avg_separation'], row['targeted_receiver'], fontsize=11, fontweight='bold')
        # Separation Specialist Quadrant (Top Left)
        if row['avg_tcr'] < 3.0 and row['avg_separation'] > 2.5:
             plt.text(row['avg_tcr'] + 0.02, row['avg_separation'], row['targeted_receiver'], fontsize=11, fontweight='bold')
        # Deep Threat Quadrant (Bottom Right)
        if row['avg_tcr'] > 3.7 and row['avg_separation'] < 2.0:
             plt.text(row['avg_tcr'] + 0.02, row['avg_separation'], row['targeted_receiver'], fontsize=11, fontweight='bold')
             
    # --- Aesthetics and Labels ---
    plt.title('Receiver Performance Matrix (Full 2023 Season)', fontsize=24, fontweight='bold', pad=20)
    plt.xlabel('Target Convergence Rate (Yds/Sec) -> [Faster to the Spot]', fontsize=16)
    plt.ylabel('Average Separation at Catch (Yards) -> [More Open]', fontsize=16)
    
    # Add average lines
    plt.axhline(qualified_players_season['avg_separation'].mean(), ls='--', color='grey', lw=2)
    plt.axvline(qualified_players_season['avg_tcr'].mean(), ls='--', color='grey', lw=2)
    
    # Customize legend
    h, l = plot.get_legend_handles_labels()
    plt.legend(h[1:7], l[1:7], title='Number of Targets', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
    
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

else:
    print("ERROR: The 'qualified_players_season' DataFrame was not found. Please re-run Step 6.")

### Final Observations: The Full-Season Receiver Performance Matrix

The full-season analysis, culminating in the Performance Matrix visualization, provides a definitive and deeply insightful look at receiver archetypes across the NFL. By plotting every qualified receiver based on our two custom metrics, we can move beyond simple leaderboards and understand the *style* and *effectiveness* of each player.

**Analysis of the Player Archetypes:**

The chart clearly segments players into distinct, meaningful categories based on their performance relative to the league averages (the dashed lines).

*   **The Separation Specialists (Top-Left Quadrant):** This quadrant is populated by the league's most reliable route runners and zone-finders. Players like **Wan'Dale Robinson**, **Tyler Conklin**, and **Gerald Everett** live here. They excel at creating space through precise routes and finding openings in coverage, even if they don't possess elite top-end speed. Our metrics successfully identify these players as masters of their craft.

*   **The Deep Threats (Bottom-Right Quadrant):** This is the home of the burners. As expected, **Tyreek Hill** is a prominent resident, alongside other explosive players like **Jayden Reed**, **Tutu Atwell**, and **Tre Tucker**. Their defining trait is an elite Target Convergence Rate, allowing them to win on deep routes. Their lower average separation confirms that they often win by outrunning defenders to a spot, leading to more tightly contested situations downfield. The model's ability to cluster these specific players together is a powerful validation of our `avg_tcr` metric.

*   **The Elite All-Rounder (Top-Right Quadrant):** This quadrant is reserved for the rare players who are significantly above average in *both* creating separation *and* getting to the spot quickly. The standout performer in this category is **Trey McBride**. Our analysis identifies him not just as a good tight end, but as a uniquely dynamic weapon who combines the separation skills of a possession receiver with the speed of a downfield threat.

### Final Project Conclusion

This project successfully developed and validated a novel framework for analyzing NFL player movement during pass plays. By leveraging the provided ball landing coordinates, we engineered two powerful, custom metrics: **Average Separation at Catch** and **Target Convergence Rate**.

Through a rigorous process of prototyping on a weekly sample and scaling to the entire 2023 season, we have proven that these metrics are not only statistically robust but also align strongly with real-world football knowledge. They successfully distinguished between player archetypes—from savvy route runners to elite deep threats—and identified unique talents like Trey McBride.

The final "Receiver Performance Matrix" is an intuitive and actionable tool that could provide tangible value to NFL coaches and scouts, offering a new lens through which to evaluate player skill sets. This methodology is innovative, data-driven, and directly addresses the core challenge of the 2026 Big Data Bowl, forming a complete and compelling submission.